In [ ]:
%matplotlib inline

In [1]:
import networkx as nx
import numpy as np
import random
import pandas as pd
import pickle as pkl
from scipy.spatial.distance import cosine, cdist
from tqdm import tqdm
from collections import Counter, OrderedDict, defaultdict
from itertools import combinations
from sklearn.metrics import average_precision_score

from random_steiner_tree import random_steiner_tree
from random_steiner_tree.util import from_gt

from graph_helpers import load_graph_by_name, get_edge_weights, swap_end_points, extract_nodes_from_tuples
from proba_helpers import tree_probability_gt, cascade_probability_gt, ic_cascade_probability_gt
from helpers import cascade_source, infected_nodes, l1_dist
from preprocess_graph import reverse_edge_weights
from root_sampler import build_true_root_sampler
from sample_pool import TreeSamplePool

from inference import infection_probability
from tree_stat import TreeBasedStatistics
from graph_tool.draw import graph_draw
from graph_tool import openmp_set_num_threads

from graph_tool import GraphView
from graph_tool.draw import sfdp_layout, graph_draw
from viz_helpers import lattice_node_pos

openmp_set_num_threads(1)

In [2]:
graph_name = 'flixster'
sampling_method = 'cut'
n_samples = 100

In [3]:
g = load_graph_by_name(graph_name, weighted=True)
g_rev = load_graph_by_name(graph_name, weighted=True, suffix='_reversed')

load graph from data/flixster/graph_weighted.gt
load graph from data/flixster/graph_weighted_reversed.gt


In [4]:
if False:
    g_und = GraphView(g, directed=False)
    g_und.set_directed(False)

    if False:
        if graph_name == 'lattice-1024':
            pos = lattice_node_pos(g, shape=(32, 32))
        else:
            pos = sfdp_layout(g)

In [5]:
p = get_edge_weights(g)
p_rev = get_edge_weights(g_rev)
if False:
    for e in g.edges():
        u, v = int(e.source()), int(e.target())
        if u < v:
            assert p[g.edge(u, v)] == p_rev[g_rev.edge(v, u)]

In [6]:
X, c = pkl.load(open('cascade-weighted/{}-mic-s0.02-oleaves/1.pkl'.format(graph_name), 'rb'))

In [7]:
root = cascade_source(c)
X = list(X)

In [8]:
print('root = {}'.format(root))
print('|terminals|: {}'.format(len(X)))
print('cascade size: {}'.format(len(infected_nodes(c))))

root = 5280
|terminals|: 20
cascade size: 40


In [9]:
def sampled_tree_freqs(gi, X, root, sampling_method, N):
    print('sampling steiner trees.')
    trees = [swap_end_points(random_steiner_tree(gi, X, root, method=sampling_method))
             for i in tqdm(range(N), total=N)]
    tree_freq = Counter(trees)
    return tree_freq


In [ ]:
# def one_run(g, X, root, n_samples, sampling_method, return_samples=False):
    
g = g_rev  # to reverse
use_P = False

if use_P:
    print('using P')
    cascade_probability_function = cascade_probability_gt
else:
    print('using P_new')    
    cascade_probability_function = ic_cascade_probability_gt


print('building g_nx')
g_nx = nx.DiGraph()
for e in tqdm(g.edges(), total=g.num_edges()):
    g_nx.add_edge(int(e.source()), int(e.target()))

print('building gi')    
gi = from_gt(g, get_edge_weights(g))

p = get_edge_weights(g)    

print('building p_dict')
p_dict = {tuple(map(int, [e.source(), e.target()])): p[e] for e in tqdm(g.edges(), total=g.num_edges())}

##########################
# naive approach
##########################
tree_freq = sampled_tree_freqs(gi, X, root, sampling_method, n_samples)
possible_trees = list(tree_freq.keys())

tree_probas = np.array([tree_freq[t] for t in possible_trees]) / n_samples
cascade_probas = np.array([cascade_probability_function(g, p_dict, t, g_nx, using_log=False) for t in possible_trees])
# print('cascade_probas', cascade_probas)
cascade_probas /= cascade_probas.sum()

# print('cascade_probas.sum()', cascade_probas.sum())
# print('tree_probas.sum()', tree_probas.sum())


# evaluation
cos_sim_only = 1 - cosine(tree_probas, cascade_probas)
l1_dist_only = l1_dist(tree_probas, cascade_probas)

del tree_probas, cascade_probas, tree_freq, possible_trees

##########################
# now we do the re-sampling
##########################
pool = TreeSamplePool(g, n_samples, sampling_method, 
                      gi=gi, 
                      with_resampling=True,
                      true_casacde_proba_func=cascade_probability_function,
                      return_type='tuples')

pool.fill(X, root_sampler=build_true_root_sampler(c))
resampled_trees = pool.samples
trees = pool._old_samples

possible_trees = list(set(trees))
# print('num. possible_trees', len(possible_trees))

resampled_tree_freq = Counter(resampled_trees)
resampled_tree_probas = np.array([resampled_tree_freq[t] for t in possible_trees]) / n_samples
print('num. unique resampled trees', len(resampled_tree_freq))
print('top frequencies', list(sorted(resampled_tree_freq.values(), reverse=True))[:10])

# here we calculate the probas based on g
# because edges point towards root
cascade_probas = np.array([cascade_probability_function(g, p_dict, t, g_nx, using_log=False) for t in possible_trees])
cascade_probas /= cascade_probas.sum()

# print('cascade_probas.sum()', cascade_probas.sum())
# print('resampled_tree_probas.sum()', resampled_tree_probas.sum())

# evaluation
cos_sim_together = 1 - cosine(resampled_tree_probas, cascade_probas)
l1_dist_together = l1_dist(resampled_tree_probas,
                           cascade_probas)


# summary
ans = OrderedDict()
ans['cos_sim_without_resampling'] = cos_sim_only
ans['l1_dist_without_resampling'] = l1_dist_only
ans['cos_sim_with_resampling'] = cos_sim_together
ans['l1_dist_with_resampling'] = l1_dist_together

# if not return_samples:
#     return ans
# else:
#     return ans, trees, resampled_trees
ans

  3%|▎         | 13008/375926 [00:00<00:02, 130077.02it/s]

using P_new
building g_nx


100%|██████████| 375926/375926 [00:03<00:00, 123990.19it/s]


building gi


  2%|▏         | 7188/375926 [00:00<00:05, 71878.18it/s]

building p_dict


100%|██████████| 375926/375926 [00:05<00:00, 67528.62it/s]

In [ ]:
top_trees_and_freq = resampled_tree_freq.most_common(10)
for tree, freq in top_trees_and_freq:
    print('tree size/freq', len(tree), freq)

In [ ]:
w = pool._sampling_weights / pool._sampling_weights.sum()

In [ ]:
t = pool._old_samples[w.argmax()]
ns = list(extract_nodes_from_tuples(t))
out_deg[ns].mean()

In [ ]:
len(X)

In [ ]:
len(infected_nodes(c))

In [ ]:
outputs/

In [ ]:
##########
# for P
##########
s = pd.Series(list(map(len, pool.samples)))
s.hist(bins=30)

In [ ]:
out_deg = g.degree_property_map('out', weight=p).a
pd.Series(out_deg[out_deg < 2]).hist(bins=30)

In [ ]:
##########
# for **P prime***
##########
trees = [swap_end_points(random_steiner_tree(gi, X, root, method=sampling_method))
         for i in range(n_samples)]
s = pd.Series(list(map(len, trees)))
s.hist(bins=30)

In [ ]:
from viz_helpers import tree_plot_setting, visualize


top_trees = Counter(pool.samples).most_common(3)


for t, f in top_trees:
    print('frequency/size', f, len(t))
    setting = tree_plot_setting(g, c, X, t)
    visualize(g_und, pos,
              **setting)

In [ ]:
estimator = TreeBasedStatistics(g, pool.samples)

probas = infection_probability(g, X, pool, estimator)

In [ ]:
from viz_helpers import InfectionProbabilityViz
viz = InfectionProbabilityViz(g_und, pos)
viz.plot(c, X, probas)


In [ ]:
pd.Series(list(map(len, pool.samples))).hist

In [ ]:
ans, trees, resampled_trees = one_run(g_rev, X, root, n_samples, sampling_method,
                                      return_samples=True)

In [ ]:
pd.Series(list(map(len, trees))).hist()

In [ ]:
len(infected_nodes(c))

In [ ]:
len(X)

In [ ]:
pd.Series(list(map(len, resampled_trees))).hist()

In [ ]:
ans